---
title: "Modeling Medical Expenditures Using Pandas And PyStan"
output: html_document
categories: statistics health-economics tutorials python
---

### Introduction
This post is basically an opportunity for me to move outside of my 'R' comfort zone and push a little bit into Python. In it, I'll cover how to 

### Step 1: Bringing in and cleaning the data.
Step 1, always, is to get the data together and ready for analysis, which involves knowing what dat you actually want to use.

For this exercise we'll be using results from the Medical Expenditure Panel Survey (MEPS). MEPS is an annual survey, financed by the US Agency for Healthcare Research and Quality (AHRQ), that uses a nationally representative sample of households to survey about (you guessed it) annual medical expenditures. There's a panel aspect to the data which just means that some households are surveyed over multiple years. We'll ignore that here and just treat the data as if it was from a single cross-section.

For actually getting the data, I cheated a bit and got data through the great `lodown` R package, which gives users easy access to many different publically-available survey datasets. Once th data was download into an .Rds file I went ahead and converted to csv which is where we'll start here.

In [22]:
import pandas as pd
import matplotlib
import pystan

meps_15 = pd.read_csv("~/Desktop/MEPS/MEPS_2015_full.csv")

print(meps_15.shape)

(35427, 1831)


Ok. 15.8k rows with 3.5k columns-a pretty wide set of data. 

We don't need all of the data-let's subset only the few columns we're interested in to make it easier to work with. Let's say we'll be modeling expenditures as a function of age, health insurance status, and presence of various chronix conditions.


In [31]:
meps_sub = meps_15[['age15x', 'totexp15','insurc15','chddx','angidx','midx','strkdx','emphdx',
                    'cancerdx','diabdx','arthdx','asthdx']]
print(meps_sub.head())

   age15x  totexp15  insurc15  chddx  angidx  midx  strkdx  emphdx  cancerdx  \
0      53     46612         2      1       1     1       2       1         2   
1      56      9207         2      2       2     2       2       2         2   
2      30         0         1      2       2     2       2       2         2   
3      23       808         2      2       2     2       2       2         2   
4       3      2721         2     -1      -1    -1      -1      -1        -1   

   diabdx  arthdx  asthdx  
0       1       1       2  
1       1       1       1  
2       2       2       2  
3       2       2       2  
4      -1      -1       2  


['duid', 'pid', 'dupersid', 'panel', 'yearind', 'saqrds24', 'all5rds', 'died', 'inst', 'military', 'entrsrvy', 'leftus', 'other', 'famid1', 'famid2', 'famid3', 'famid4', 'famid5', 'famidy1', 'famidy2', 'famidyr1', 'famidyr2', 'cpsfamy1', 'cpsfamy2', 'fcszy1', 'fcszy2', 'fcrpy1', 'fcrpy2', 'ruletr1', 'ruletr2', 'ruletr3', 'ruletr4', 'ruletr5', 'ruletry1', 'ruletry2', 'rusize1', 'rusize2', 'rusize3', 'rusize4', 'rusize5', 'rusizey1', 'rusizey2', 'ruclas1', 'ruclas2', 'ruclas3', 'ruclas4', 'ruclas5', 'ruclasy1', 'ruclasy2', 'famsze1', 'famsze2', 'famsze3', 'famsze4', 'famsze5', 'famszey1', 'famszey2', 'fmrsy1', 'fmrsy2', 'famsy1', 'famsy2', 'famszyr1', 'famszyr2', 'famrfpy1', 'famrfpy2', 'region1', 'region2', 'region3', 'region4', 'region5', 'regiony1', 'regiony2', 'refprs1', 'refprs2', 'refprs3', 'refprs4', 'refprs5', 'refprsy1', 'refprsy2', 'resp1', 'resp2', 'resp3', 'resp4', 'resp5', 'respy1', 'respy2', 'proxy1', 'proxy2', 'proxy3', 'proxy4', 'proxy5', 'proxyy1', 'proxyy2', 'intvlang',